### import libraries and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../data/poverty/train.csv')
test = pd.read_csv('../data/poverty/test.csv')

In [3]:
data_dict = pd.read_table('../data/poverty/data_dict.txt',header = None)

In [4]:
data_dict = data_dict[0].str.split(pat = ',',expand = True)

In [5]:
data_dict.set_index(0,inplace = True)

In [6]:
data_dict.head()

,1,2,3,4
0,,,,
v2a1,Monthly rent payment,None,None,None
hacdor,=1 Overcrowding by bedrooms,None,None,None
rooms,number of all rooms in the house,None,None,None
hacapo,=1 Overcrowding by rooms,None,None,None
v14a,=1 has bathroom in the household,None,None,None


In [7]:
train.shape

(9557, 143)

In [8]:
train.set_index('Id',inplace  = True)
test.set_index('Id',inplace  = True)

### how do we want to split up our data types?

#### credit to https://www.kaggle.com/willkoehrsen/start-here-a-complete-walkthrough who has determined the data types and how they should be used

In [9]:
id_ = ['idhogar']

target = ['Target']

ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone']

ind_ordered = ['rez_esc', 'escolari', 'age']

hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding']

sqr_ = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 
        'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq']

#### tidy up object data types

In [10]:
test['dependency'].value_counts()

yes          5388
no           4289
.5           3678
2            1769
1.5          1758
.33333334    1533
.66666669    1130
8            1037
.25           684
3             596
1.3333334     278
2.5           224
.2            216
.75           203
4             195
.40000001     175
.60000002     128
1.6666666     120
5              96
.16666667      56
1.25           54
.80000001      45
.14285715      32
2.3333333      30
.83333331      22
3.5            18
7              16
.85714287      13
2.25           13
3.3333333      13
1.2            11
.375           11
.2857143        9
.125            9
6               7
Name: dependency, dtype: int64

In [11]:
mapping = {'yes' : 1, 'no' : 0}

train['dependency'] = train['dependency'].replace(mapping).astype(np.float64)
train['edjefe'] = train['edjefe'].replace(mapping).astype(np.float64)
train['edjefa'] = train['edjefa'].replace(mapping).astype(np.float64)

In [12]:
test['dependency'] = test['dependency'].replace(mapping).astype(np.float64)
test['edjefe'] = test['edjefe'].replace(mapping).astype(np.float64)
test['edjefa'] = test['edjefa'].replace(mapping).astype(np.float64)

#### missing values

In [13]:
miss_vals = train[ind_bool + ind_ordered + hh_bool + hh_ordered + hh_cont + sqr_].isnull().sum()

miss_vals[miss_vals != 0]

rez_esc      7928
v18q1        7342
v2a1         6860
meaneduc        5
SQBmeaned       5
dtype: int64

In [14]:
miss_values_index = miss_vals[miss_vals != 0].index.values.tolist()

pd.options.display.max_colwidth = 1000

data_dict.loc[miss_values_index,:1]

,1
0,
rez_esc,Years behind in school
v18q1,number of tablets household owns
v2a1,Monthly rent payment
meaneduc,average years of education for adults (18+)
SQBmeaned,square of the mean years of education of adults (>=18) in the household


In [15]:
train.drop(columns = ['rez_esc','v18q1','v2a1'],inplace = True)
test.drop(columns = ['rez_esc','v18q1','v2a1'],inplace = True)

#remove the values from the indexes that we have previous got rid of

ind_ordered.remove('rez_esc')
hh_ordered.remove('v18q1')
hh_cont.remove('v2a1')

### creating percentage columns

In [16]:
new_cols = []

In [17]:
def calculate_percentages(num,denom):
    if denom == 0:
        return 0
    else:
        percentage = num/denom
        return percentage

In [18]:
train['r4h1_r4h3'] = train.apply(lambda x : calculate_percentages(x['r4h1'],x['r4h3']),axis = 1)
train['r4m1_r4m3'] = train.apply(lambda x : calculate_percentages(x['r4m1'],x['r4m3']),axis = 1)
train['r4t1_r4t3'] = train.apply(lambda x : calculate_percentages(x['r4t1'],x['r4t3']),axis = 1)

In [19]:
test['r4h1_r4h3'] = test.apply(lambda x : calculate_percentages(x['r4h1'],x['r4h3']),axis = 1)
test['r4m1_r4m3'] = test.apply(lambda x : calculate_percentages(x['r4m1'],x['r4m3']),axis = 1)
test['r4t1_r4t3'] = test.apply(lambda x : calculate_percentages(x['r4t1'],x['r4t3']),axis = 1)

In [20]:
new_cols.extend(['r4h1_r4h3','r4m1_r4m3','r4t1_r4t3'])

In [21]:
train['nin%'] = train.apply(lambda x : calculate_percentages(x['hogar_nin'],x['hogar_total']),axis = 1)
train['adul%'] = train.apply(lambda x : calculate_percentages(x['hogar_adul'],x['hogar_total']),axis = 1)
train['mayor%'] = train.apply(lambda x : calculate_percentages(x['hogar_mayor'],x['hogar_total']),axis = 1)

In [22]:
test['nin%'] = test.apply(lambda x : calculate_percentages(x['hogar_nin'],x['hogar_total']),axis = 1)
test['adul%'] = test.apply(lambda x : calculate_percentages(x['hogar_adul'],x['hogar_total']),axis = 1)
test['mayor%'] = test.apply(lambda x : calculate_percentages(x['hogar_mayor'],x['hogar_total']),axis = 1)

In [23]:
new_cols.extend(['nin%','adul%','mayor%'])

### Try a neural network

In [24]:
train[ind_bool + ind_ordered + hh_bool + hh_ordered + hh_cont + sqr_ + new_cols].dtypes.value_counts()

int64      129
float64     14
dtype: int64

In [25]:
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [26]:
scaler = StandardScaler()
imputer = Imputer()


imputer = Imputer()

X = imputer.fit_transform(train[ind_bool + ind_ordered + hh_bool + hh_ordered + hh_cont + sqr_ + new_cols])
X_test_final = imputer.transform(test[ind_bool + ind_ordered + hh_bool + hh_ordered + hh_cont + sqr_ + new_cols])

X = scaler.fit_transform(X)
X_test_final = scaler.transform(X_test_final)

y = train['Target']

In [27]:
#create train and validation set

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33,random_state = 42)

In [101]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=12, ratio = 'auto')
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

unique, counts = np.unique(y_train_res, return_counts=True)
np.asarray((unique, counts)).T

array([[   1, 4052],
       [   2, 4052],
       [   3, 4052],
       [   4, 4052]], dtype=int64)

In [102]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001
                    ,solver='sgd',learning_rate = 'adaptive',verbose=True,random_state=21,tol=0.000000001)

In [103]:
clf.fit(X_train,y_train)

Iteration 1, loss = 1.16052371
Iteration 2, loss = 1.01562551
Iteration 3, loss = 0.96726434
Iteration 4, loss = 0.93376959
Iteration 5, loss = 0.91188290
Iteration 6, loss = 0.89581232
Iteration 7, loss = 0.88328110
Iteration 8, loss = 0.87530851
Iteration 9, loss = 0.86924849
Iteration 10, loss = 0.86020777
Iteration 11, loss = 0.85341960
Iteration 12, loss = 0.84686663
Iteration 13, loss = 0.84076911
Iteration 14, loss = 0.83558883
Iteration 15, loss = 0.83149585
Iteration 16, loss = 0.82561695
Iteration 17, loss = 0.81988865
Iteration 18, loss = 0.81542516
Iteration 19, loss = 0.81303249
Iteration 20, loss = 0.80749387
Iteration 21, loss = 0.80213439
Iteration 22, loss = 0.79821828
Iteration 23, loss = 0.79523393
Iteration 24, loss = 0.78971683
Iteration 25, loss = 0.78523554
Iteration 26, loss = 0.78168428
Iteration 27, loss = 0.77708695
Iteration 28, loss = 0.77321199
Iteration 29, loss = 0.77009296
Iteration 30, loss = 0.76684083
Iteration 31, loss = 0.76251961
Iteration 32, los

Iteration 254, loss = 0.17867113
Iteration 255, loss = 0.14538792
Iteration 256, loss = 0.14036908
Iteration 257, loss = 0.13891822
Iteration 258, loss = 0.13699452
Iteration 259, loss = 0.13617392
Iteration 260, loss = 0.13516825
Iteration 261, loss = 0.13466606
Iteration 262, loss = 0.13415114
Iteration 263, loss = 0.13330557
Iteration 264, loss = 0.13273520
Iteration 265, loss = 0.13241092
Iteration 266, loss = 0.13340679
Iteration 267, loss = 0.13200729
Iteration 268, loss = 0.13099051
Iteration 269, loss = 0.13054499
Iteration 270, loss = 0.13065254
Iteration 271, loss = 0.13031048
Iteration 272, loss = 0.12951205
Iteration 273, loss = 0.13049526
Iteration 274, loss = 0.12902475
Iteration 275, loss = 0.12852754
Iteration 276, loss = 0.12783513
Iteration 277, loss = 0.12754645
Iteration 278, loss = 0.12705979
Iteration 279, loss = 0.12670468
Iteration 280, loss = 0.12616714
Iteration 281, loss = 0.12565229
Iteration 282, loss = 0.12528130
Iteration 283, loss = 0.12497306
Iteration 

Iteration 500, loss = 0.10535804


C:\Users\jberry2\Anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=21, shuffle=True,
       solver='sgd', tol=1e-09, validation_fraction=0.1, verbose=True,
       warm_start=False)

In [104]:
y_pred = clf.predict(X_test)

In [105]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score

accuracy_score(y_test,y_pred)

0.7733037412809132

In [106]:
f1_score(y_test,y_pred,average = 'macro')

0.6528572546328564

### create test file of predictions

In [34]:
clf.predict(X_test_final)

array([4, 4, 4, ..., 2, 1, 2], dtype=int64)

In [35]:
test.index

Index(['ID_2f6873615', 'ID_1c78846d2', 'ID_e5442cf6a', 'ID_a8db26a79',
       'ID_a62966799', 'ID_e77d38d45', 'ID_3c5f4bd51', 'ID_a849c29bd',
       'ID_472fa82da', 'ID_24864adcc',
       ...
       'ID_f0c9c06f7', 'ID_4b7feead3', 'ID_c2650e696', 'ID_64958963c',
       'ID_ecdf63132', 'ID_a065a7cad', 'ID_1a7c6953b', 'ID_07dbb4be2',
       'ID_34d2ed046', 'ID_34754556f'],
      dtype='object', name='Id', length=23856)

In [36]:
submission = pd.DataFrame(data = clf.predict(X_test_final),index=test.index,columns = ['Target'])
submission.to_csv('submission.csv')

In [37]:
submission['Target'].value_counts(normalize = True)

4    0.652582
2    0.170062
3    0.112508
1    0.064847
Name: Target, dtype: float64

### oversampling

### debugging of first model

In [39]:
results = confusion_matrix(y_test,y_pred)

results

array([[ 128,   55,   25,   56],
       [  30,  330,   45,  119],
       [  23,   46,  217,  136],
       [  32,   86,   62, 1764]], dtype=int64)

In [55]:
results = pd.DataFrame(data = np.stack((np.array(y_test),y_pred),axis = 1)
            ,index = y_test.index,columns = ['Actual','Prediction'])

In [61]:
results['Error'] = results['Actual'] == results['Prediction']

In [65]:
results = results[results['Error'] == False]

In [67]:
train.merge(results,left_index=True,right_index = True)

,hacdor,rooms,hacapo,v14a,refrig,v18q,r4h1,r4h2,r4h3,r4m1,...,Target,r4h1_r4h3,r4m1_r4m3,r4t1_r4t3,nin%,adul%,mayor%,Actual,Prediction,Error
Id,,,,,,,,,,,,,,,,,,,,,
ID_279628684,0,3,0,1,1,0,0,1,1,0,...,4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4,2,False
ID_5c837d8a4,0,4,0,1,1,1,0,1,1,0,...,4,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,4,1,False
ID_422226730,0,5,0,1,1,0,0,1,1,0,...,4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4,3,False
ID_be1c0c929,0,5,0,1,1,0,0,4,4,0,...,4,0.000000,0.000000,0.000000,0.200000,0.800000,0.000000,4,3,False
ID_57b05a403,0,5,0,1,1,0,0,4,4,0,...,4,0.000000,0.000000,0.000000,0.200000,0.800000,0.000000,4,3,False
ID_eb1d477b3,0,5,0,1,1,0,0,1,1,1,...,2,0.000000,0.333333,0.250000,0.500000,0.500000,0.000000,2,4,False
ID_a0fb0a5c4,0,1,0,0,0,0,0,1,1,0,...,2,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2,3,False
ID_3f2e2fa81,0,4,0,1,1,0,1,2,3,0,...,4,0.333333,0.000000,0.200000,0.400000,0.600000,0.000000,4,2,False
ID_fdf18bbee,0,5,0,1,1,0,0,1,1,0,...,2,0.000000,0.000000,0.000000,0.000000,1.000000,0.200000,2,4,False
